## Compute source time course (STC) from Raw and Epochs data object and save as pickle for later analyses
#### Input: *_eyes_open-raw.fif, *_eyes_closed-raw.fif, *-epo.fif
####  Output: *_label_ts.pkl

In [1]:
import os
from pathlib import Path
import sys
sys.path.append('/home/wanglab/Documents/George Kenefati/Code/eeg_toolkit/')
import os
import mne
from eeg_toolkit import utils, preprocess, source_localization

0 files missing from root.txt in /home/wanglab/mne_data/MNE-fsaverage-data
0 files missing from bem.txt in /home/wanglab/mne_data/MNE-fsaverage-data/fsaverage


### Paths

In [2]:
# Time window
times_tup,time_win_path = preprocess.get_time_window(5)

# Average dipoles, Always true
average_dipoles = True

# Save inverse
save_inv = False

# Save stc as MAT
save_stc_mat = True

[-2.5,0.0,2.5]


In [3]:
# Read in processed raw data
data_dir = Path("../../Data")
processed_data_path = data_dir / 'Processed Data'
epo_path = data_dir / time_win_path

### Subject IDs

In [21]:
# viable subjects
sub_ids = utils.import_subs(os.path.join(data_dir),'sub_ids.txt')
# select only 64ch subs and turn into set
sub_ids = sub_ids[7:]
print(sub_ids)

['018', '020', '021', '022', '023', '024', '027', '029', '030', '031', '032', '033', '034', '035', '036', '037', '038', '039', '040', '041', '042', '043', '044', '045', '046', '048', '049', '050', '051', '052', '053', '054', '055', '056', '057', '058', 'C1.', 'C2.', 'C3.', 'C5.', 'C6.', 'C7.', 'C8.', 'C9.', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C24', 'C25', 'C26', 'C27']


In [5]:
print(f"Chronics: {len([el for el in sub_ids if el.startswith('0')])}")
print(f"Controls: {len([el for el in sub_ids if el.startswith('C')])}")
print(f"Total: {len(sub_ids)}")

Chronics: 36
Controls: 25
Total: 61


#### Look for subjects who do not have EO or EC cropped data

In [6]:
# For edge cases of subjects missing eyes open or eyes closed data
no_eyes_open = []
no_eyes_closed = []

for sub_id in sub_ids:
    if not os.path.exists(os.path.join(processed_data_path, f"{sub_id}_eyes_closed-raw.fif")):
        no_eyes_closed.append(sub_id)
        print(f"Subject: {sub_id} missing eyes closed data")
    if not os.path.exists(os.path.join(processed_data_path, f"{sub_id}_eyes_open-raw.fif")):
        no_eyes_open.append(sub_id)
        print(f"Subject: {sub_id} missing eyes open data")
        
print(no_eyes_open)
print(no_eyes_closed)

Subject: 045 missing eyes closed data
Subject: 045 missing eyes open data
Subject: C1. missing eyes closed data
Subject: C1. missing eyes open data
Subject: C5. missing eyes closed data
Subject: C5. missing eyes open data
Subject: C8. missing eyes closed data
Subject: C8. missing eyes open data
Subject: C20 missing eyes closed data
Subject: C20 missing eyes open data
Subject: C21 missing eyes closed data
Subject: C21 missing eyes open data
Subject: C22 missing eyes closed data
Subject: C22 missing eyes open data
['045', 'C1.', 'C5.', 'C8.', 'C20', 'C21', 'C22']
['045', 'C1.', 'C5.', 'C8.', 'C20', 'C21', 'C22']


In [7]:
# Get stc only from selected labels
roi_names = [# Left
             'rostralanteriorcingulate-lh', # Left Rostral ACC
             'caudalanteriorcingulate-lh', # Left Caudal ACC
             'postcentral-lh', # Left S1,
             'insula-lh', 'superiorfrontal-lh', # Left Insula, Left DL-PFC,
             'medialorbitofrontal-lh', # Left Medial-OFC
             # Right
             'rostralanteriorcingulate-rh', # Right Rostral ACC
             'caudalanteriorcingulate-rh', # Right Caudal ACC
             'postcentral-rh', # , Right S1
             'insula-rh', 'superiorfrontal-rh', # Right Insula, Right DL-PFC
             'medialorbitofrontal-rh'] # Right Medial-OFC

### Compute STCs

In [8]:
import numpy as np

In [9]:
method = 'MNE'
# Paths
stc_path = (data_dir / f'Source Time Courses ({method})')
EO_resting_save_path = stc_path / "Eyes Open"
EC_resting_save_path = stc_path / "Eyes Closed"
zscored_epochs_save_path = stc_path / "zscored_Epochs" / time_win_path
save_paths = [EO_resting_save_path,EC_resting_save_path,zscored_epochs_save_path] 

In [27]:
nan_subjects=[]

methods = ['MNE','dSPM']

sub_ids = [sub for sub in sub_ids if sub not in no_eyes_open and sub not in no_eyes_closed]

for save_stc_mat in [False]:#True, False]:    
    for sub_id in sub_ids: 
        for method in methods:
            for path,condition in zip(save_paths, ['eyes_open','eyes_closed','epochs']):
                print(f"Source Localization Method: {method}\n Condition: {path}")               

                # read pickled data
                label_ts = utils.unpickle_data(str(os.path.join(path, f"{sub_id}_{condition}.pkl")))
                
                # Check if nan
                if np.isnan(label_ts).any():
                    print(sub_id)


Source Localization Method: MNE
 Condition: ../../Data/Source Time Courses (MNE)/Eyes Open
Source Localization Method: MNE
 Condition: ../../Data/Source Time Courses (MNE)/Eyes Closed
Source Localization Method: MNE
 Condition: ../../Data/Source Time Courses (MNE)/zscored_Epochs/5_sec_time_window
Source Localization Method: dSPM
 Condition: ../../Data/Source Time Courses (MNE)/Eyes Open
Source Localization Method: dSPM
 Condition: ../../Data/Source Time Courses (MNE)/Eyes Closed
Source Localization Method: dSPM
 Condition: ../../Data/Source Time Courses (MNE)/zscored_Epochs/5_sec_time_window
Source Localization Method: MNE
 Condition: ../../Data/Source Time Courses (MNE)/Eyes Open
Source Localization Method: MNE
 Condition: ../../Data/Source Time Courses (MNE)/Eyes Closed
Source Localization Method: MNE
 Condition: ../../Data/Source Time Courses (MNE)/zscored_Epochs/5_sec_time_window
Source Localization Method: dSPM
 Condition: ../../Data/Source Time Courses (MNE)/Eyes Open
Source Loca